# Learning `Message` in `Lanchain` using a `Chatbot`

## Install Libraries

In [1]:
!pip install --upgrade pip
!pip install --upgrade transformers accelerate

# Import Libraries

In [2]:
import torch

In [25]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Load Model

## Download Model

In [4]:
model_id = "Qwen/Qwen3-4B-Instruct-2507"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load model with quantization
raw_model_llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    # load_in_4bit = True if torch.cuda.is_available() else False,
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" ,  # <-- IMPORTANT for Colab Free (CPU),
    offload_folder="offload",  # Will offload some layers to CPU
    offload_state_dict=True
)

print("Model loaded!")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Model loaded!


## Make Pipeline

In [5]:
pipe = pipeline(
    task = "text-generation",
    model = raw_model_llm, # `model_llm` should be the AutoModelForCausalLM from the previous cell
    tokenizer = tokenizer,
    # Generation parameters like max_new_tokens and temperature should be passed via model_kwargs to HuggingFacePipeline
)

model_llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={
        "max_new_tokens": 20,
        "temperature": 0.2 # Corrected typo from temparature
    }
)

Device set to use cuda:0


In [6]:
model_llm.invoke("What is the Capital of Bangladesh?")

'What is the Capital of Bangladesh? The capital of Bangladesh is **Dhaka**. It is also the largest city and a major economic, cultural, and educational center of the country. Dhaka serves as the seat of the government and is home to the national parliament, the president\'s office, and numerous government institutions. \n\nWhile Bangladesh has a capital city, it is important to note that the country does not have a "capital" in the traditional sense of a geographical or administrative center in the same way as some other nations. However, **Dhaka** is universally recognized as the de facto capital. \n\nSo, the answer is: **Dhaka**. ✅'

# Chat Model Build

In [7]:
model = ChatHuggingFace(llm=model_llm, verbose=True)

In [8]:
%%time
result = model.invoke("What is the Capital of Bangladesh?")
result.content

CPU times: user 960 ms, sys: 0 ns, total: 960 ms
Wall time: 1.1 s


'<|im_start|>user\nWhat is the Capital of Bangladesh?<|im_end|>\n<|im_start|>assistant\nThe capital of Bangladesh is Dhaka.'

# Message

# ChatBot v1

In [9]:
while True:
  input_str = input("You: ")

  if input_str == 'exit':
    break

  result = model.invoke(input_str)
  print("AI: " + result.content)
  print('-'*25)

You: how are you?
AI: <|im_start|>user
how are you?<|im_end|>
<|im_start|>assistant
I'm doing great, thank you! 🌟 I love chatting with people and helping out. How can I assist you today? 😊
-------------------------
You: which one is biggest 2 and 20
AI: <|im_start|>user
which one is biggest 2 and 20<|im_end|>
<|im_start|>assistant
The biggest number between **2** and **20** is **20**. ✅

Because 20 is greater than 2.
-------------------------
You: multiply bigger number with 10
AI: <|im_start|>user
multiply bigger number with 10<|im_end|>
<|im_start|>assistant
Sure! Please provide the two numbers you'd like me to multiply the bigger one by 10.  

For example, if the numbers are 7 and 15, I would multiply 15 (the bigger number) by 10 to get 150.  

Just give me the numbers, and I’ll do the calculation! 😊
-------------------------
You: exit


## Chatbot v2

In [11]:
message = ["You are simple ai assistant"]

while True:
  input_str = input("You: ")
  message.append(input_str)

  if input_str == 'exit':
    break

  result = model.invoke(message)
  print("AI: " + result.content)
  print('-'*25)
  message.append(result.content)

print('*'*30)
print(message)
print('*'*30)

You: which one is bigger 2 or 10?
AI: <|im_start|>user
You are simple ai assistant<|im_end|>
<|im_start|>user
which one is bigger 2 or 10?<|im_end|>
<|im_start|>assistant
10 is bigger than 2. 😊
-------------------------
You: multiply bigger number with 10
AI: <|im_start|>user
You are simple ai assistant<|im_end|>
<|im_start|>user
which one is bigger 2 or 10?<|im_end|>
<|im_start|>user
<|im_start|>user
You are simple ai assistant<|im_end|>
<|im_start|>user
which one is bigger 2 or 10?<|im_end|>
<|im_start|>assistant
10 is bigger than 2. 😊<|im_end|>
<|im_start|>user
multiply bigger number with 10<|im_end|>
<|im_start|>assistant
The bigger number is 10.  

Now, multiply 10 by 10:  
10 × 10 = 100 ✅  

Answer: **100** 😊
-------------------------
You: exit
******************************
['You are simple ai assistant', 'which one is bigger 2 or 10?', '<|im_start|>user\nYou are simple ai assistant<|im_end|>\n<|im_start|>user\nwhich one is bigger 2 or 10?<|im_end|>\n<|im_start|>assistant\n10 is

## Chatbot v3

In [13]:
message = [
    SystemMessage(content="You are simple ai assistant")
]

while True:
  input_str = input("You: ")
  message.append(HumanMessage(content=input_str))

  if input_str == 'exit':
    break

  result = model.invoke(message)
  print("AI: " + result.content)
  print('-'*25)
  message.append(AIMessage(content=result.content))

print('*'*30)
print(message)
print('*'*30)

You: hi how are you?
AI: <|im_start|>system
You are simple ai assistant<|im_end|>
<|im_start|>user
hi how are you?<|im_end|>
<|im_start|>assistant
Hi! I'm doing great, thanks for asking! 😊 How are you doing today?
-------------------------
You: 2 vs 10 which one is biggest?
AI: <|im_start|>system
You are simple ai assistant<|im_end|>
<|im_start|>user
hi how are you?<|im_end|>
<|im_start|>assistant
<|im_start|>system
You are simple ai assistant<|im_end|>
<|im_start|>user
hi how are you?<|im_end|>
<|im_start|>assistant
Hi! I'm doing great, thanks for asking! 😊 How are you doing today?<|im_end|>
<|im_start|>user
2 vs 10 which one is biggest?<|im_end|>
<|im_start|>assistant
10 is bigger than 2. 😊
-------------------------
You: multiply bigger number with 10


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


AI: <|im_start|>system
You are simple ai assistant<|im_end|>
<|im_start|>user
hi how are you?<|im_end|>
<|im_start|>assistant
<|im_start|>system
You are simple ai assistant<|im_end|>
<|im_start|>user
hi how are you?<|im_end|>
<|im_start|>assistant
Hi! I'm doing great, thanks for asking! 😊 How are you doing today?<|im_end|>
<|im_start|>user
2 vs 10 which one is biggest?<|im_end|>
<|im_start|>assistant
<|im_start|>system
You are simple ai assistant<|im_end|>
<|im_start|>user
hi how are you?<|im_end|>
<|im_start|>assistant
<|im_start|>system
You are simple ai assistant<|im_end|>
<|im_start|>user
hi how are you?<|im_end|>
<|im_start|>assistant
Hi! I'm doing great, thanks for asking! 😊 How are you doing today?<|im_end|>
<|im_start|>user
2 vs 10 which one is biggest?<|im_end|>
<|im_start|>assistant
10 is bigger than 2. 😊<|im_end|>
<|im_start|>user
multiply bigger number with 10<|im_end|>
<|im_start|>assistant
The bigger number is 10.  

Multiply it by 10:  
10 × 10 = **100** 🎉
--------------

# List of Message with Dynamic Prompts

In [22]:
list_chat_templates = ChatPromptTemplate([
    ('system', "You are a {role} ai assistant"),
    ('human', " {a} vs {b} which one is biggest?")
])

In [23]:
list_chat_prompts = list_chat_templates.invoke({
    'role': 'mathatician',
    'a': 10,
    'b': 5
})

In [24]:
list_chat_prompts

ChatPromptValue(messages=[SystemMessage(content='You are a mathatician ai assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content=' 10 vs 5 which one is biggest?', additional_kwargs={}, response_metadata={})])

# Mesage-Place-Holder

- fetch history from database
- use it to current conversation

In [31]:
chats_template = ChatPromptTemplate([
    ('system', "You are a {role} ai assistant"),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', " {operation} result with {a}")
])

## History Fetch

In [32]:
chat_history = []
with open('chat_history.txt') as f:
  chat_history.extend(f.readlines())
chat_history

["SystemMessage(content='You are simple ai assistant', additional_kwargs={}, response_metadata={})\n",
 "HumanMessage(content='hi how are you?', additional_kwargs={}, response_metadata={})\n",
 'AIMessage(content="<|im_start|>system\\nYou are simple ai assistant<|im_end|>\\n<|im_start|>user\\nhi how are you?<|im_end|>\\n<|im_start|>assistant\\nHi! I\'m doing great, thanks for asking! 😊 How are you doing today?", additional_kwargs={}, response_metadata={})\n',
 "HumanMessage(content='2 vs 10 which one is biggest?', additional_kwargs={}, response_metadata={})\n",
 'AIMessage(content="<|im_start|>system\\nYou are simple ai assistant<|im_end|>\\n<|im_start|>user\\nhi how are you?<|im_end|>\\n<|im_start|>assistant\\n<|im_start|>system\\nYou are simple ai assistant<|im_end|>\\n<|im_start|>user\\nhi how are you?<|im_end|>\\n<|im_start|>assistant\\nHi! I\'m doing great, thanks for asking! 😊 How are you doing today?<|im_end|>\\n<|im_start|>user\\n2 vs 10 which one is biggest?<|im_end|>\\n<|im_s

## Make Prompts

In [35]:
prompts = chats_template.invoke({
    'a': 7,
    'operation': 'multiply',
    'role': 'mathematician',
    'chat_history': chat_history
})

In [36]:
prompts

ChatPromptValue(messages=[SystemMessage(content='You are a mathematician ai assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content="SystemMessage(content='You are simple ai assistant', additional_kwargs={}, response_metadata={})\n", additional_kwargs={}, response_metadata={}), HumanMessage(content="HumanMessage(content='hi how are you?', additional_kwargs={}, response_metadata={})\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='AIMessage(content="<|im_start|>system\\nYou are simple ai assistant<|im_end|>\\n<|im_start|>user\\nhi how are you?<|im_end|>\\n<|im_start|>assistant\\nHi! I\'m doing great, thanks for asking! 😊 How are you doing today?", additional_kwargs={}, response_metadata={})\n', additional_kwargs={}, response_metadata={}), HumanMessage(content="HumanMessage(content='2 vs 10 which one is biggest?', additional_kwargs={}, response_metadata={})\n", additional_kwargs={}, response_metadata={}), HumanMessage(content='AIMessage(content

## Get AI Result

In [37]:
placeholder_result = model.invoke(prompts)
placeholder_result.content

'<|im_start|>system\nYou are a mathematician ai assistant<|im_end|>\n<|im_start|>user\nSystemMessage(content=\'You are simple ai assistant\', additional_kwargs={}, response_metadata={})\n<|im_end|>\n<|im_start|>user\nHumanMessage(content=\'hi how are you?\', additional_kwargs={}, response_metadata={})\n<|im_end|>\n<|im_start|>user\nAIMessage(content="<|im_start|>system\\nYou are simple ai assistant<|im_end|>\\n<|im_start|>user\\nhi how are you?<|im_end|>\\n<|im_start|>assistant\\nHi! I\'m doing great, thanks for asking! 😊 How are you doing today?", additional_kwargs={}, response_metadata={})\n<|im_end|>\n<|im_start|>user\nHumanMessage(content=\'2 vs 10 which one is biggest?\', additional_kwargs={}, response_metadata={})\n<|im_end|>\n<|im_start|>user\nAIMessage(content="<|im_start|>system\\nYou are simple ai assistant<|im_end|>\\n<|im_start|>user\\nhi how are you?<|im_end|>\\n<|im_start|>assistant\\n<|im_start|>system\\nYou are simple ai assistant<|im_end|>\\n<|im_start|>user\\nhi how a

**Now, multiplying that result (100) by 7:\n\n100 × 7 = **700**\n\n✅ Final answer: **700** 😊**